In [3]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torchvision
import numpy as np
import pandas as pd
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
dry_bean_dataset = fetch_ucirepo(id=602)

# data (as pandas dataframes)
features = dry_bean_dataset.data.features
labels = dry_bean_dataset.data.targets

# Lable Encoding
label_encoder = LabelEncoder()
x = label_encoder.fit_transform(labels)

# converting the lables to tensor
tensor_labels = torch.tensor(x, dtype=torch.float32)

# Encoding the dataset to extract the features out of it
print(tensor_labels)

features_train, features_test, labels_train, labels_test = train_test_split(
    features, x, test_size=0.2, random_state=42)

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, features, labels):
        self.x = torch.tensor(features.values, dtype=torch.float32)
        self.y = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        return self.x[index], self.y[index]

# Create instances of your custom datasets
train_dataset = CustomImageDataset(features_train, labels_train)
test_dataset = CustomImageDataset(features_test, labels_test)

# Create custom data loaders for training and test sets
batch_size = 32
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
class MultiLayerPerceptron(nn.Module):
    def __init__(self):
        super(MultiLayerPerceptron, self).__init__()
        self.layer = nn.Linear(16, 32)
        self.second_layer = nn.Linear(32, 7)
        self.relu = nn.ReLU()
        self.second_relu = nn.ReLU()

    def forward(self, tensor):
        output = self.layer(tensor)
        output = self.relu(output)
        output = self.second_layer(output)
        output = self.second_relu(output)
        return output


model = MultiLayerPerceptron()

In [ ]:
# Define loss function and optimizer
epochs = 100
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

for epoch in range(epochs):
    model.train()
    for inputs, labels in train_data_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

In [ ]:
# Evaluation
model.eval()
with torch.no_grad():
    test_outputs = model()
    test_loss = criterion(test_outputs, test_dataset.y)
    predicted_labels = torch.argmax(test_outputs, dim=1)

# set the model to evaluation mode
model.eval()

# initialize variables to store the validation loss and accuracy for this epoch
test_loss = 0.0
test_correct = 0
test_total = 0
# evaluate the model on the validation data
with torch.no_grad():
    for X, y in test_data_loader:
        outputs = model(X)
        loss = criterion(outputs, y.long())

        test_loss += loss.item() * X.size(0)
        _, predicted = torch.max(outputs.data, 1)
        test_total += y.size(0)
        test_correct += (predicted == y).sum().item()

# calculate the test loss and accuracy for this epoch
test_loss /= len(test_data_loader.dataset)
test_acc = test_correct / test_total

# print the test loss and accuracy for this epoch
print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}')